In [41]:
import sys,os
import trimesh as tri
import numpy as np
import torch

from utils import mesh_processing, lddmm_utils

GPU = True
device_idx = 0
torch.cuda.get_device_name(device_idx)

if GPU:
    device = torch.device("cuda:"+str(device_idx) if torch.cuda.is_available() else "cpu")
else:
    device = torch.device("cpu")
print(device)

cuda:0


In [42]:
root_path = "../datasets/COMA/"

idents = ['FaceTalk_170725_00137_TA', 'FaceTalk_170728_03272_TA', 'FaceTalk_170731_00024_TA', 'FaceTalk_170809_00138_TA',
          'FaceTalk_170811_03274_TA', 'FaceTalk_170811_03275_TA', 'FaceTalk_170904_00128_TA', 'FaceTalk_170904_03276_TA',
          'FaceTalk_170908_03277_TA', 'FaceTalk_170912_03278_TA', 'FaceTalk_170913_03279_TA', 'FaceTalk_170915_00223_TA']

exprs = ['bareteeth', 'cheeks_in', 'eyebrow', 'high_smile', 'lips_back', 'lips_up',
         'mouth_down', 'mouth_extreme', 'mouth_middle', 'mouth_open', 'mouth_side', 'mouth_up']

In [70]:
path_id1_expr1 = root_path + idents[0] + '/' + exprs[0]
list_id1_expr1 = os.listdir(path_id1_expr1)
list_id1_expr1.sort()

path_id2_expr1 = root_path + idents[1] + '/' + exprs[0]
list_id2_expr1 = os.listdir(path_id1_expr1)
list_id2_expr1.sort()

In [71]:
id1_neutral = tri.load(path_id1_expr1 + '/' + list_id1_expr1[0])
id1_expr1 = tri.load(path_id1_expr1 + '/' + list_id1_expr1[len(list_id1_expr1)//2])


id2_neutral = tri.load(path_id2_expr1 + '/' + list_id2_expr1[0])
id2_expr1 = tri.load(path_id2_expr1 + '/' + list_id2_expr1[len(list_id2_expr1)//2])

In [74]:
def varifold_dist(mesh1, mesh2, sigma):
    torchdtype = torch.float
    sigma = torch.tensor([sigma], dtype=torchdtype, device=device)
    
    # PyKeOps counterpart
    KeOpsdeviceId = device.index  # id of Gpu device (in case Gpu is  used)
    KeOpsdtype = torchdtype.__str__().split(".")[1]  # 'float32'
    
    V1, F1 = torch.Tensor(mesh1.vertices).to(dtype=torchdtype, device=device).requires_grad_(True), torch.Tensor(mesh1.faces).to(dtype=torch.int32, device=device)
    V2, F2 = torch.Tensor(mesh2.vertices).to(dtype=torchdtype, device=device).requires_grad_(True), torch.Tensor(mesh2.faces).to(dtype=torch.int32, device=device)
    
    L = lddmm_utils.lossVarifoldSurf(F1, V2, F2, lddmm_utils.GaussLinKernel(sigma=sigma))(V1)
    
    return L.sqrt().item()

In [78]:
varifold_dist(id1_neutral, id1_expr1, sigma = 0.1)

0.0027023139409720898